In [1]:
!pip install spacy


In [13]:
import os #when loading paths
import pandas as pd 
import spacy#for tokenization
import torch
from torch.nn.utils.rnn import pad_sequence#pad batch
from torch.utils.data import DataLoader, Dataset
from PIL import Image#load image
import torchvision.transforms as transforms
import numpy as np


# we want to convert text -> numberical values 
#1. we need a vocaburary mapping each word to a index
#2. we need to setup a pytorch dataset to load the data
#3. setup padding of every batch( all example should be of same seq_len and setup data loader)
spacy_eng= spacy.load("en")
class Vocabulary:
    def __init__(self,freq_threshold):
        self.itos = {0:"<PAD>",1:"<SOS>",2:"<EOS>",3:"<UNK>"}
        self.stoi = {"<PAD>":0,"<SOS>":1,"<EOS>":2,"<UNK>":3}
        self.freq_threshold=freq_threshold
    def __len__(self):
        return len(self.itos)
    @staticmethod
    def tokenizer_eng(text):
        return [tok.text for tok in spacy_eng.tokenizer(str(text))]
    def build_vocabulary(self,sentence_list):
        frequencies = {}
        idx = 4
        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):
                if word not in frequencies:
                    frequencies[word] = 1
                else:
                    frequencies[word] += 1
                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] =word 
                    idx+=1
                    
    def numericallize(self,text):
        torkenized_text = self.tokenizer_eng(text)
        return[ self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]for token in torkenized_text]
    
class FlickrDataset(Dataset):
    def __init__(self,root_dir,captions_file,freq_threshold=4,transform=None):
        self.root_dir=root_dir
        self.df=pd.read_csv(captions_file,error_bad_lines=False)
        self.transform = transform
        self.img = self.df['image_name']
        self.captions=self.df['comment']
        self.vocab=Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.captions.tolist())
    def __len__(self):
        return len(self.df)
    def __getitem__(self,index):
        caption=self.captions[index]
        imag_id=self.img[index]
        foldername= 'E:/Tensorflow_in_practice/imagecaptioning/flickr30k_images/flickr30k_images/'
        img = Image.open(os.path.join(self.root_dir,foldername,imag_id)).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption +=self.vocab.numericallize(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])
        return [img, torch.tensor(numericalized_caption)]
class MyCollate:
    def __init__(self,pad_idx):
        self.pad_idx=pad_idx
    def __call__(self,batch):
        loader = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor()])  
        print(np.array(batch[0][0]).shape)
        imgs = [loader(item[0]).unsqueeze(0) for item in batch]
        imgs =torch.cat(imgs,dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets,batch_first=False, padding_value=self.pad_idx)
        return imgs,targets
def get_loader(roo_folder,
               annoation_csv,
               transform,
               batch_size=32,
               num_workers=8,
               shuffle=True,
               pin_memory=True):
    dataset=FlickrDataset(roo_folder,annoation_csv,transform=transform)
    pad_idx = dataset.vocab.stoi['<PAD>']
    loader = DataLoader(dataset=dataset,batch_size=batch_size,shuffle=shuffle,pin_memory=pin_memory,collate_fn=MyCollate(pad_idx=pad_idx))
    return loader,dataset
if __name__ == "__main__":
    transform = transforms.Compose(
        [transforms.Resize((224, 224)), transforms.ToTensor(),]
    )
    path = 'E:/Tensorflow_in_practice/imagecaptioning/flickr30k_images/flickr30k_images/'
    annotation_path = 'E:/tensorflow_in_practice/imagecaptioning/flickr30k_images/results.csv'
    dataloader,dataset = get_loader(roo_folder=path, annoation_csv=annotation_path,transform=None)         
    for idx, (imgs, captions) in enumerate(dataloader):
        print(imgs.shape)
        print(captions.shape)

(375, 500, 3)
torch.Size([32, 3, 224, 224])
torch.Size([35, 32])
(375, 500, 3)
torch.Size([32, 3, 224, 224])
torch.Size([27, 32])
(375, 500, 3)
torch.Size([32, 3, 224, 224])
torch.Size([38, 32])
(390, 500, 3)
torch.Size([32, 3, 224, 224])
torch.Size([23, 32])
(333, 500, 3)
torch.Size([32, 3, 224, 224])
torch.Size([29, 32])
(500, 375, 3)
torch.Size([32, 3, 224, 224])
torch.Size([25, 32])
(303, 500, 3)
torch.Size([32, 3, 224, 224])
torch.Size([32, 32])
(500, 333, 3)
torch.Size([32, 3, 224, 224])
torch.Size([24, 32])
(337, 500, 3)
torch.Size([32, 3, 224, 224])
torch.Size([25, 32])
(333, 500, 3)
torch.Size([32, 3, 224, 224])
torch.Size([26, 32])
(346, 500, 3)
torch.Size([32, 3, 224, 224])
torch.Size([25, 32])
(332, 500, 3)
torch.Size([32, 3, 224, 224])
torch.Size([25, 32])
(335, 500, 3)
torch.Size([32, 3, 224, 224])
torch.Size([27, 32])
(500, 370, 3)
torch.Size([32, 3, 224, 224])
torch.Size([35, 32])
(375, 500, 3)
torch.Size([32, 3, 224, 224])
torch.Size([31, 32])
(500, 333, 3)
torch.Size(

KeyboardInterrupt: 

In [10]:
import os  # when loading file paths
import pandas as pd  # for lookup in annotation file
import spacy  # for tokenizer
import torch
from torch.nn.utils.rnn import pad_sequence  # pad batch
from torch.utils.data import DataLoader, Dataset
from PIL import Image  # Load img
import torchvision.transforms as transforms


# We want to convert text -> numerical values
# 1. We need a Vocabulary mapping each word to a index
# 2. We need to setup a Pytorch dataset to load the data
# 3. Setup padding of every batch (all examples should be
#    of same seq_len and setup dataloader)
# Note that loading the image is very easy compared to the text!

# Download with: python -m spacy download en
spacy_eng = spacy.load("en")


class Vocabulary:
    def __init__(self, freq_threshold):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer_eng(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(str(text))]

    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4

        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):
                if word not in frequencies:
                    frequencies[word] = 1

                else:
                    frequencies[word] += 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]


class FlickrDataset(Dataset):
    def __init__(self, root_dir, captions_file, transform=None, freq_threshold=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(captions_file)
        self.transform = transform

        # Get img, caption columns
        self.imgs = self.df["image_name"]
        self.captions = self.df["comment"]

        # Initialize vocabulary and build vocab
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.captions.tolist())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        caption = self.captions[index]
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])

        return img, torch.tensor(numericalized_caption)


class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value=self.pad_idx)

        return imgs, targets


def get_loader(
    root_folder,
    annotation_file,
    transform,
    batch_size=32,
    num_workers=8,
    shuffle=True,
    pin_memory=True,
):
    dataset = FlickrDataset(root_folder, annotation_file, transform=transform)

    pad_idx = dataset.vocab.stoi["<PAD>"]

    loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=shuffle,
        pin_memory=pin_memory,
        collate_fn=MyCollate(pad_idx=pad_idx),
    )

    return loader, dataset


if __name__ == "__main__":
    transform = transforms.Compose(
        [transforms.Resize((224, 224)), transforms.ToTensor(),]
    )

    loader, dataset = get_loader(
        'E:/Tensorflow_in_practice/imagecaptioning/flickr30k_images/flickr30k_images/','E:/tensorflow_in_practice/imagecaptioning/flickr30k_images/results.csv', transform=transform
    )

    for idx, (imgs, captions) in enumerate(loader):
        print(imgs.shape)
        print(captions.shape)

BrokenPipeError: [Errno 32] Broken pipe